In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
import seaborn as sns
import numpy as np
import glob

from sklearn.externals.joblib import Memory
memory = Memory(cachedir='/tmp', verbose=0)
#@memory.cache above any def fxn.

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime

In [ ]:
#cohort import

os.chdir('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling') #use to change working directory
wd= os.getcwd() #'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling'


final_pt_df2 = pd.read_csv(Path(wd + '/data/raw/csv/01102018_final_pt_df2_v.csv') , index_col=0) #only for patients with minimum vitals
patients= list(final_pt_df2['subject_id'].unique())
hadm_id= list(final_pt_df2['hadm_id'].unique())
icustay_id= list(final_pt_df2['icustay_id'].unique())
icustay_id= [int(x) for x in icustay_id]

In [ ]:
#importing in all clinical_variable files
date= '22112018'
os.chdir(r'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/processed/')
big_df= pd.read_csv(Path(wd+'/data/processed/%s_longdf_preImp.csv' %(date),  index_col=0))
big_df= big_df.reset_index(drop=True).iloc[:,1:]

In [ ]:
big_df.head()

In [5]:
big_df.groupby('label')['value'].describe().sort_values('unique')
##categorical variables:
#rrt, phenylephrine, norepinephrine, vasopressin, dobutamine, dopamine, epinephrine, leukocyte, vent_recieved


In [ ]:
big_df[big_df['label']=='vent_recieved']['value'].value_counts() #
#df["B"] = df["A"].astype('category')

In [7]:
big_df.loc[big_df.loc[:,'label']=='vent_recieved','value']= big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'].astype('category')

time: 1.41 s


In [8]:
#big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'].apply(astype('category'))
big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'] = pd.Categorical(big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'])

time: 1.45 s


In [9]:
big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'].describe()

count     14668
unique        3
top        None
freq       8387
Name: value, dtype: object

time: 479 ms


In [10]:
hi_value= ['bands',
'bilirubin',
'bun',
'chloride',
'creatinine',
'daily_sofa',
'glucose',
'heartrate',
'inr',
'lactate',
'pco2',
'potassium',
'ptt',
'resprate',
'temperature',
'weight', 'rrt', 'phenylephrine', 'norepinephrine', 'vasopressin', 'dobutamine', 'dopamine', 'epinephrine']

low_value=['bicarbonate',
'diasbp',
'hemoglobin',
'meanartpress',
'mingcs',
'ph',
'platelet',
'spo2',
'sysbp',
'pao2/fio2']

both_value=['calcium',
'sodium',
'wbc']

time: 5.82 ms


In [11]:
#lambda x: x if not np.isnan(x) else -1

dict1={str(x):max for x in hi_value}
dict2={str(x):min for x in low_value } #dict2={str(x):min for x in low_value }
dict3={str(x):[max,min] for x in both_value }

time: 1.91 ms


In [12]:
def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

dict1 = merge_two_dicts(dict1, dict2)
dict1 = merge_two_dicts(dict1, dict3)

time: 3.52 ms


In [13]:
len(dict1.keys())

36

time: 1.78 ms


### to do: need to convert all dtypes to a numeric type taht pereserves nan. 

In [14]:
categorical=['leukocyte','nitrite','vent_recieved']

big_categorical= big_df.loc[big_df.loc[:,'label'].isin(categorical),:]
big_noCat= big_df.loc[big_df.loc[:,'label'].isin(dict1.keys()),:]
big_noCat['value']= big_noCat['value'].apply(pd.to_numeric, args=('coerce',)) #instead of convert to float, may preserve nan's better. 


time: 2min 36s


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [17]:
big_max= big_noCat.loc[big_noCat.loc[:,'label'].isin(hi_value),:]
table = pd.pivot_table(big_max, values='value', columns='label', index=['icustay_id'],aggfunc=max, dropna=False)

time: 2.17 s


In [18]:
big_min= big_noCat.loc[big_noCat.loc[:,'label'].isin(low_value),:]
table2 = pd.pivot_table(big_min, values='value', columns='label', index=['icustay_id'],aggfunc=min, dropna=False)

time: 1.19 s


In [19]:
big_both= big_noCat.loc[big_noCat.loc[:,'label'].isin(both_value),:]
table3 = pd.pivot_table(big_both, values='value', columns='label', index=['icustay_id'],aggfunc=[max,min], dropna=False)

time: 330 ms


In [20]:
print(len(table),len(table2),len(table3))

14668 14668 14618
time: 1.01 ms


In [21]:
worst_df=pd.merge(table.reset_index(), table2.reset_index(), how='left')
worst_df=pd.merge(worst_df, table3.reset_index(), left_on='icustay_id', right_on='icustay_id',how='left')

time: 51.5 ms


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandas/core/generic.py:3111: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [28]:
print(len(list(worst_df)), #40- icustay_id - label = 38. with 

big_df['label'].nunique(),
     
set(big_df['label'].unique())- set(worst_df)   )


40 39 {'calcium', 'wbc', 'nitrite', 'vent_recieved', 'sodium', 'leukocyte'}
time: 582 ms


In [23]:
big_max= big_df.loc[big_df.loc[:,'label'].isin(hi_value),:]

time: 602 ms


In [29]:
big_categorical.head()

,icustay_id,t_0,value,label,uom,delta,source,subject_id
0,200012.0,2153-12-23,Neg/Not_tested,leukocyte,pos/neg category,0 days 00:00:00.000000000,leuk,28448
1,200012.0,2153-12-23,Neg/Not_tested,nitrite,pos/neg category,0 days 00:00:00.000000000,nit,28448
2,200012.0,2153-12-23,None,vent_recieved,mech/O2/none category,0 days 00:00:00.000000000,ventcategory,28448
85,200014.0,2105-02-17,Neg/Not_tested,leukocyte,pos/neg category,0 days 00:00:00.000000000,leuk,9514
86,200014.0,2105-02-17,Neg/Not_tested,nitrite,pos/neg category,0 days 00:00:00.000000000,nit,9514


time: 10.2 ms


In [32]:
big_categorical= big_categorical.pivot(
    index='icustay_id',
    values='value',
    columns='label') #need to convert to long format. should be one row per patient per time. 

time: 20.7 ms


In [33]:
worst_df=pd.merge(worst_df, big_categorical, left_on='icustay_id', right_on='icustay_id',how='left')

time: 23 ms


In [38]:
worst_df['leukocyte']=worst_df['leukocyte'].astype("category")
worst_df['nitrite']=worst_df['nitrite'].astype("category")
worst_df['vent_recieved']=worst_df['vent_recieved'].astype("category")

time: 7.9 ms


In [41]:
worst_df

,icustay_id,bands,bilirubin,bun,chloride,creatinine,daily_sofa,dobutamine,dopamine,epinephrine,...,sysbp,"(max, calcium)","(max, sodium)","(max, wbc)","(min, calcium)","(min, sodium)","(min, wbc)",leukocyte,nitrite,vent_recieved
0,200012.0,11.0,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,...,95.0,NaN,NaN,4.9,NaN,NaN,4.9,Neg/Not_tested,Neg/Not_tested,None
1,200014.0,NaN,NaN,24.0,114.0,0.8,2.0,0.0,0.0,0.0,...,70.0,1.24,143.0,14.3,1.13,138.0,10.7,Neg/Not_tested,Neg/Not_tested,Mech
2,200019.0,NaN,NaN,24.0,109.0,1.0,6.0,0.0,0.0,0.0,...,69.0,NaN,140.0,13.5,NaN,140.0,12.8,Neg/Not_tested,Neg/Not_tested,Mech
3,200030.0,NaN,14.5,28.0,111.0,1.0,4.0,1.0,0.0,0.0,...,107.0,1.33,141.0,13.9,1.15,138.0,12.5,Neg/Not_tested,Neg/Not_tested,Oxygen
4,200033.0,NaN,NaN,16.0,108.0,0.7,1.0,0.0,0.0,0.0,...,99.0,1.06,145.0,7.7,1.03,133.0,6.2,Neg/Not_tested,Neg/Not_tested,None
5,200036.0,NaN,NaN,31.0,110.0,1.0,1.0,0.0,0.0,0.0,...,96.0,NaN,141.0,6.7,NaN,136.0,4.3,Neg/Not_tested,Neg/Not_tested,Oxygen
6,200055.0,NaN,0.3,40.0,100.0,8.9,8.0,0.0,0.0,0.0,...,53.0,1.11,140.0,6.3,1.11,137.0,5.7,Neg/Not_tested,Neg/Not_tested,None
7,200059.0,NaN,0.7,50.0,109.0,1.5,6.0,0.0,0.0,0.0,...,68.0,1.17,140.0,14.8,1.01,132.0,8.5,Neg/Not_tested,Neg/Not_tested,None
8,200061.0,NaN,5.9,15.0,106.0,0.9,3.0,0.0,0.0,0.0,...,124.0,NaN,142.0,15.6,NaN,137.0,11.5,Neg/Not_tested,Neg/Not_tested,Oxygen
9,200063.0,NaN,0.8,54.0,108.0,2.3,6.0,0.0,0.0,0.0,...,85.0,1.20,143.0,8.1,1.00,129.0,7.5,Neg/Not_tested,Neg/Not_tested,Mech


time: 76.2 ms


In [42]:
date= '22112018'
pd.DataFrame(worst_df).to_csv(Path(
    wd+'/data/processed/%s_worstdf_preImp.csv' %(date)))

time: 528 ms


In [ ]:
# big_categorical["icustay_id"].isin(icustay_id)

In [ ]:
#big_noCat['value']=big_noCat['value'].astype(float, copy=False)
#big_noCat.groupby(['icustay_id','label'], as_index=False)['value'].agg(dict1) #

In [ ]:
# test= big_noCat.groupby(['icustay_id'], as_index=False)['value'].agg(dict1)
# table = pd.pivot_table(big_noCat, values='value', columns='label', index=['icustay_id'],aggfunc=max, dropna=False)
# table

# pd.pivot_table(df,index=["Manager","Status"],columns=["Product"],values=["Quantity","Price"],
#                aggfunc={"Quantity":len,"Price":np.sum},fill_value=0)

In [ ]:
# #table = pivot_table(df, values=['D', 'E'], index=['A', 'C'],aggfunc={})
# ##don't know why this doesn't work. 
# table = pd.pivot_table(big_noCat, values='value',columns='label', index='icustay_id',aggfunc=dict1, dropna=False)
# table


In [ ]:
# big_agg= big_df.loc[big_df.loc[:,'label'].isin(non_category),:].groupby(['icustay_id','label'], as_index=False)['value'].apply(lambda x: float(x))

    #.agg({str(x):['max'] for x in hi_value})
#big_agg_count= big_agg.reset_index().pivot(index='icustay_id',columns='label', values=0)#, levels='icustay_id')

In [ ]:
# hi_df= big_df.loc[big_df.loc[:,'label'].isin(hi_value),:]
# hi_df['value']=hi_df['value'].astype(float, copy=False)
# hi_df.groupby(['icustay_id','label'], as_index=False)['value'].agg('max') #this works

In [ ]:
# %whos

# del big_agg
# del hi_df
# del final_pt_df2